In [1]:
import socket
import struct
import pickle
import threading
import tkinter as tk
from tkinter import messagebox
import cv2
from PIL import Image, ImageTk

class ServerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Server with Dynamic Image Display")

        # 左邊的結束按鈕
        self.end_button = tk.Button(self.root, text="結束", command=self.quit_server)
        self.end_button.pack(side=tk.LEFT, padx=10, pady=10)

        # 右邊顯示影像的畫布
        self.canvas = tk.Canvas(self.root, width=800, height=800, bg="black")
        self.canvas.pack(side=tk.RIGHT, padx=10, pady=10)

        self.client_images = []  # 儲存來自客戶端的影像
        self.running = True

        # 開啟 socket server 的線程
        server_thread = threading.Thread(target=self.start_server)
        server_thread.start()

    def start_server(self):
        server_ip = '192.168.50.36'  # 本地測試 IP
        server_port = 5555

        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_socket.bind((server_ip, server_port))
        server_socket.listen(10)
        print(f"Server is listening on {server_ip}:{server_port}")

        while self.running:
            client_socket, addr = server_socket.accept()
            print(f"Connection from: {addr}")
            # 為每個客戶端創建一個新的線程來接收影像
            client_thread = threading.Thread(target=self.handle_client, args=(client_socket,))
            client_thread.start()

    def handle_client(self, client_socket):
        data = b""
        payload_size = struct.calcsize("I")  # 用來接收消息大小

        while self.running:
            try:
                # 接收消息大小
                while len(data) < payload_size:
                    packet = client_socket.recv(4096)
                    if not packet:
                        print("Client disconnected")
                        return
                    data += packet

                packed_msg_size = data[:payload_size]
                data = data[payload_size:]
                msg_size = struct.unpack("I", packed_msg_size)[0]

                # 接收影像數據
                while len(data) < msg_size:
                    data += client_socket.recv(4096)

                frame_data = data[:msg_size]
                data = data[msg_size:]

                # 解壓縮並反序列化影像數據
                compressed_frame = pickle.loads(frame_data)
                frame = cv2.imdecode(compressed_frame, cv2.IMREAD_COLOR)

                # 添加進 client_images 列表
                self.client_images.append(frame)
                if len(self.client_images) > 5:
                    self.client_images.pop(0)  # 保持最多5個影像

                # 更新畫布
                self.update_canvas()

            except Exception as e:
                print(f"Error: {e}")
                break

        client_socket.close()

    def update_canvas(self):
        self.canvas.delete("all")  # 清除畫布中的舊影像
        num_clients = len(self.client_images)

        if num_clients == 0:
            return

        # 計算每個影像應該顯示的大小
        grid_size = int(num_clients ** 0.5) + 1
        block_size = min(800 // grid_size, 800 // grid_size)

        for i, img in enumerate(self.client_images):
            # Resize 每個影像
            resized_img = cv2.resize(img, (block_size, block_size))

            # 將 OpenCV 影像轉為 PIL 格式
            img_rgb = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb)
            img_tk = ImageTk.PhotoImage(img_pil)

            # 計算每個影像應該放置的 x, y 位置
            row, col = divmod(i, grid_size)
            x = col * block_size
            y = row * block_size

            # 在畫布上顯示影像
            self.canvas.create_image(x, y, anchor=tk.NW, image=img_tk)
            self.canvas.image = img_tk  # 保存對象引用，防止被垃圾回收

    def quit_server(self):
        self.running = False
        self.root.quit()


# 創建應用主窗口
root = tk.Tk()
app = ServerApp(root)
root.mainloop()


Server is listening on 192.168.50.36:5555
Connection from: ('192.168.50.227', 35822)
Connection from: ('192.168.50.191', 53234)
Client disconnected
Connection from: ('192.168.50.191', 35174)
Error: main thread is not in main loop
Error: main thread is not in main loop
